$\textbf{PROGRAMMING ASSIGNMENT}$
---
This assignment is a test of your algorithmic thinking. You can use any A.I. tools ot assist you in this coding assignment.

Instructions:

- This is done by pair, preferably your thesis partner.
- Each person should create a Github Repo titled 'Assignment_1_Data_Analytics'.
- Read the journal provided.
- Develop a Python implementation of the procedures in the journal.
- Deadline is before premidterm week.

In [7]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import mahalanobis
from scipy.stats import norm, wilcoxon
import pulp
import matplotlib.pyplot as plt

# Simulate longitudinal data for 400 patients
np.random.seed(42)
n_patients = 400
time_points = np.arange(0, 25, 3)  # 0-24 months in 3-month intervals

# Generate patient data
patients = []
for pid in range(n_patients):
    base_pain = np.random.randint(0, 10)
    base_urgency = np.random.randint(0, 10)
    base_freq = np.random.randint(0, 10)
    
    # Simulate time-varying symptoms with autoregressive process
    pain = [base_pain + np.random.randint(-1, 2)]
    urgency = [base_urgency + np.random.randint(-1, 2)]
    freq = [base_freq + np.random.randint(-1, 2)]
    for t in range(1, len(time_points)):
        pain.append(pain[-1] + np.random.randint(-1, 2))
        urgency.append(urgency[-1] + np.random.randint(-1, 2))
        freq.append(freq[-1] + np.random.randint(-1, 2))
    
    # Clip to 0-9 scale
    pain = np.clip(pain, 0, 9)
    urgency = np.clip(urgency, 0, 9)
    freq = np.clip(freq, 0, 9)
    
    print(time_points)
    # Treatment assignment (hazard increases with recent symptom worsening)
    treat_prob = 0.1 + 0.05*(np.mean(pain[-2:]) > 5)  # Simplified model
    treated = np.random.binomial(1, treat_prob)
    treat_time = np.random.choice(time_points[1:-1]) if treated else np.inf
    
    patients.append({
        'id': pid,
        'treated': treated,
        'treat_time': treat_time,
        'base_pain': base_pain,
        'base_urgency': base_urgency,
        'base_freq': base_freq,
        'time_pain': pain,
        'time_urgency': urgency,
        'time_freq': freq,
    })

df = pd.DataFrame(patients)

[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 12 15 18 21 24]
[ 0  3  6  9 1

In [8]:
df['treat_time'].value_counts()

treat_time
inf     347
3.0      11
6.0      10
21.0      9
18.0      7
12.0      6
15.0      5
9.0       5
Name: count, dtype: int64

In [11]:
# Add this before matching:
df['time_pain_latest'] = df['time_pain'].apply(lambda x: x[-1] if isinstance(x, list) else x)
df['time_urgency_latest'] = df['time_urgency'].apply(lambda x: x[-1] if isinstance(x, list) else x)
df['time_freq_latest'] = df['time_freq'].apply(lambda x: x[-1] if isinstance(x, list) else x)

In [12]:
def balanced_risk_set_matching(df):
    treated = df[df['treated'] == 1].sort_values('treat_time')
    controls = df[df['treated'] == 0].copy()
    pairs = []
    
    # Define covariates (use scalar fields)
    covariates = ['base_pain', 'base_urgency', 'base_freq', 
                 'time_pain_latest', 'time_urgency_latest', 'time_freq_latest']
    
    # Calculate quantiles
    quantiles = {col: np.quantile(df[col], [0.33, 0.67]) for col in covariates}
    
    # Create binary indicators for quantile groups
    for col in covariates:
        for i, q in enumerate(quantiles[col]):
            df[f'{col}_q{i+1}'] = (df[col] > q).astype(int)
    
    # Initialize optimization problem
    prob = pulp.LpProblem("BalancedMatching", pulp.LpMinimize)
    
    # Create decision variables
    edges = []
    for _, t_row in treated.iterrows():
        t_time = t_row['treat_time']
        # Ensure treat_time is a scalar in controls
        eligible = controls[controls['treat_time'].astype(float) > t_time]
        for _, c_row in eligible.iterrows():
            edges.append((t_row['id'], c_row['id']))
    
    x = pulp.LpVariable.dicts("pair", edges, cat='Binary')
    
    # Objective: Minimize Mahalanobis distance
    cov_matrix = np.cov(df[covariates].T)
    inv_cov = np.linalg.inv(cov_matrix)
    distances = {}
    for (t, c) in edges:
        t_vec = df.loc[df['id'] == t, covariates].values.ravel()
        c_vec = df.loc[df['id'] == c, covariates].values.ravel()
        distances[(t, c)] = mahalanobis(t_vec, c_vec, inv_cov)
    
    prob += pulp.lpSum([distances[e] * x[e] for e in edges])
    
    # Balance constraints
    balance_vars = [col for col in df.columns if '_q' in col]
    for col in balance_vars:
        treated_total = df[df['treated'] == 1][col].sum()
        control_expr = pulp.lpSum([df.loc[df['id'] == c, col].values[0] * x[(t, c)] 
                                 for (t, c) in edges])
        prob += control_expr == treated_total
    
    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=False))
    
    # Extract matches
    matched_pairs = []
    for (t, c) in edges:
        if x[(t, c)].value() > 0.5:
            matched_pairs.append((t, c))
    
    return matched_pairs

In [13]:
matched_pairs = balanced_risk_set_matching(df)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
def assess_balance(df, matched_pairs):
    treated_ids = [p[0] for p in matched_pairs]
    control_ids = [p[1] for p in matched_pairs]
    
    treated = df[df['id'].isin(treated_ids)]
    controls = df[df['id'].isin(control_ids)]
    
    # Standardized differences
    balance_report = {}
    for col in ['base_pain', 'base_urgency', 'base_freq']:
        mean_diff = treated[col].mean() - controls[col].mean()
        pooled_std = np.sqrt((treated[col].var() + controls[col].var())/2)
        balance_report[col] = mean_diff / pooled_std
    
    # Distribution comparison
    plt.figure(figsize=(10, 6))
    for i, col in enumerate(['base_pain', 'time_pain'], 1):
        plt.subplot(1, 2, i)
        plt.hist(treated[col], alpha=0.5, label='Treated')
        plt.hist(controls[col], alpha=0.5, label='Control')
        plt.title(col)
    plt.legend()
    plt.show()
    
    return balance_report

balance = assess_balance(df, matched_pairs)

In [ ]:
def calculate_contrasts(df, matched_pairs):
    contrasts = []
    for t, c in matched_pairs:
        treated = df[df['id'] == t].iloc[0]
        control = df[df['id'] == c].iloc[0]
        
        # Find 3-month post-treatment outcome
        t_idx = np.where(time_points == treated['treat_time'])[0][0]
        t_post = treated['time_pain'][t_idx+1] if t_idx+1 < len(time_points) else np.nan
        
        # Control outcome at same relative time
        c_post = control['time_pain'][t_idx+1] if t_idx+1 < len(time_points) else np.nan
        
        if not np.isnan(t_post) and not np.isnan(c_post):
            baseline_diff = (treated['base_pain'] + treated['time_pain'][t_idx])/2
            contrast = (t_post - baseline_diff) - (c_post - control['base_pain'])
            contrasts.append(contrast)
    
    return contrasts

contrasts = calculate_contrasts(df, matched_pairs)

# Wilcoxon signed-rank test
_, pval = wilcoxon(contrasts)
print(f"Wilcoxon p-value: {pval:.4f}")

# Trimean calculation
def trimean(data):
    q1, med, q3 = np.quantile(data, [0.25, 0.5, 0.75])
    return (q1 + 2*med + q3)/4

print(f"Contrast trimean: {trimean(contrasts):.2f}")

In [ ]:
def sensitivity_analysis(contrasts, gamma_values):
    results = {}
    for gamma in gamma_values:
        # Calculate worst-case p-value bounds
        ranks = np.argsort(np.abs(contrasts))[::-1] + 1
        p_plus = 1 / (1 + gamma)
        p_minus = gamma / (1 + gamma)
        
        # Calculate sensitivity bound
        t_plus = sum(ranks * p_plus)
        t_minus = sum(ranks * p_minus)
        
        # Normal approximation
        mu = (t_plus + t_minus)/2
        sigma = np.sqrt((t_plus - mu)**2/3)
        z = (sum(ranks) - mu)/sigma
        p_bound = norm.sf(abs(z))
        
        results[gamma] = p_bound
    
    # Plot sensitivity
    plt.plot(results.keys(), results.values(), marker='o')
    plt.axhline(0.05, color='red', linestyle='--')
    plt.xlabel('Gamma (Γ)')
    plt.ylabel('Upper Bound p-value')
    plt.show()
    
    return results

sensitivity = sensitivity_analysis(contrasts, [1.0, 1.5, 2.0, 2.5, 3.0])